In [ ]:
import os

import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split

import cv2
from google.cloud import storage
from kaggle_datasets import KaggleDatasets
from random import seed, randint, random, choice
from PIL import Image
import tensorflow_addons as tfa

In [ ]:
strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 8
GCS_PATH = KaggleDatasets().get_gcs_path('plant-pathology-2021-fgvc8')

In [ ]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy


def build_decoder(with_labels=True, target_size=(256, 256), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        print(img.shape)

        degrees = randint(-4, 4)
        img = tfa.image.transform_ops.rotate(img, degrees * 10 * 3.14 / 180.0)
        
        csize = randint(int(image_size * 0.5), image_size)
        img = tf.image.random_crop(img, [csize, csize, 3], seed = 16)
        img = tf.image.resize(img, (image_size, image_size), method='nearest')
        
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=18700, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

In [ ]:
load_dir = "/kaggle/input/plant-pathology-2021-fgvc8/"
df = pd.read_csv(load_dir + 'train.csv')

class_name = df.labels.unique().tolist()

print(class_name)
num_classes = len(class_name)
print(num_classes)

df.labels = df.labels.astype(str)
labels = tf.keras.utils.to_categorical([class_name.index(x) for x in df.labels])

paths = GCS_PATH+'/train_images/'+df['image']

In [ ]:
train_paths, valid_paths, train_labels, valid_labels = sklearn.model_selection.train_test_split(
    paths, labels, train_size=0.8, shuffle=True, random_state=123)

In [ ]:
image_size = 224

decoder = build_decoder(with_labels=True, target_size=(image_size, image_size))

train_dataset = build_dataset(
    train_paths, train_labels, bsize=BATCH_SIZE, decode_fn=decoder, augment=True
)

valid_dataset = build_dataset(
    valid_paths, valid_labels, bsize=BATCH_SIZE, decode_fn=decoder,
    repeat=False, shuffle=True, augment=True
)

In [ ]:
from tensorflow.keras import layers

def build_model():
    base_model = tf.keras.applications.MobileNet(
                        include_top=False,
                        dropout=0.5,
                        weights="imagenet",
                        input_shape=(image_size, image_size, 3))
    for layer in base_model.layers:
        layer.trainable = True

    model = tf.keras.models.Sequential([
            layers.BatchNormalization(input_shape=input_shape),
            base_model,
            layers.GlobalAveragePooling2D(),
            layers.Dense(512, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(num_classes, activation="softmax")
    ])
    return model    

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint("bestmodel_tpu.h5",
                                                save_best_only=True,
                                                save_weights_only=True,
                                                monitor='val_loss',
                                                mode='auto',
                                                verbose=1)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                 factor=0.1,
                                                 patience=3,
                                                 min_lr=1e-12,
                                                 mode='auto',
                                                 verbose=1)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                  mode='min', 
                                                  verbose=1, 
                                                  patience=5)

optimizer = tf.keras.optimizers.Nadam(lr=0.0001, 
                                      beta_1=0.9, beta_2=0.999, 
                                      epsilon=1e-7)

steps_per_epoch = train_paths.shape[0] // BATCH_SIZE
validation_steps = valid_paths.shape[0] // BATCH_SIZE

In [ ]:
with strategy.scope():
    model = build_model()

In [ ]:
epochs = 20

callbacks = [checkpoint, reduce_lr, early_stopping]

model.compile(optimizer=optimizer, 
                loss='categorical_crossentropy', 
                metrics='accuracy')

history = model.fit(train_dataset,
                    epochs=epochs,
                    verbose=1, callbacks=callbacks,
                    validation_data=valid_dataset,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps)

In [ ]:
import matplotlib.pyplot as plt
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model.load_weights('/kaggle/input/plant-pathology-e50/bestmodel_tpu.h5', by_name=True)

In [ ]:
test_dir = '/kaggle/input/plant-pathology-2021-fgvc8/test_images/'

def _parse_function(filename):
    image = tf.keras.preprocessing.image.load_img(test_dir+filename)
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = tf.keras.preprocessing.image.smart_resize(image, (image_size, image_size))
    image = tf.reshape(image, (-1, image_size, image_size, 3))
    image = tf.cast(image, tf.float32) / 255.
    return image

filenames = tf.io.gfile.listdir(test_dir)
dataset = list(map(_parse_function, filenames))
dataset = tf.data.Dataset.from_tensor_slices(dataset)
prediction = model.predict(dataset)
prediction = tf.argmax(prediction, axis=1)


submission_result = pd.DataFrame({'image' : filenames, 'labels' : prediction})
submission_result['labels'] = submission_result['labels'].apply(lambda x: class_name[int(x)])
submission_result.to_csv('submission.csv', index=False)
submission_result.head()